In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

import warnings
from pathlib import Path

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import lightgbm as lgbm

In [ ]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 데이터 로드

In [ ]:
# # 로컬 버전

# data_dir = Path('../input/dankook')
# feature_dir = Path('../output/feature')
# val_dir = Path('../output/oof_pred')
# test_dir = Path('../output/test_pred')
# sub_dir = Path('../output/sub')


# train_file = data_dir / 'train.csv'
# test_file = data_dir / 'test.csv'
# sample_file = data_dir / 'sample_submission.csv'

In [ ]:
# 코렙 

from google.colab import drive
drive.mount('/content/drive')

data_dir = Path('/content/drive/My Drive/Colab Notebooks/input/dankook')
feature_dir = Path('/content/drive/My Drive/Colab Notebooks/output/feature')
val_dir = Path('/content/drive/My Drive/Colab Notebooks/output/oof_pred')
test_dir = Path('/content/drive/My Drive/Colab Notebooks/output/test_pred')
sub_dir = Path('/content/drive/My Drive/Colab Notebooks/output/sub')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

In [ ]:
algorithm_name = 'lgbmcv'
feature_name = 'polynomial_feature'
model_name = f'{algorithm_name}_{feature_name}'

polynomial_feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
polynomial_feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
polynomial_feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'

lgbm_oof_pred_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
lgbm_oof_pred_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
lgbm_oof_pred_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'

lgbm_test_pred_ver1_file = test_dir / f'{model_name}_test_pred_ver1.csv'
lgbm_test_pred_ver2_file = test_dir / f'{model_name}_test_pred_ver2.csv'
lgbm_test_pred_ver3_file = test_dir / f'{model_name}_test_pred_ver3.csv'

lgbm_submission_ver1_file = sub_dir / f'{model_name}_submission_Ver1.csv'
lgbm_submission_ver2_file = sub_dir / f'{model_name}_submission_Ver2.csv'
lgbm_submission_ver3_file = sub_dir / f'{model_name}_submission_Ver3.csv'

In [ ]:
SEED = 2020
num_class = 3
n_splits = 5
target_column = 'class'

## Ver1 데이터 로드

In [ ]:
dataset = pd.read_csv(polynomial_feature_Ver1_file, index_col=0)
print(dataset.shape)
dataset.head()

(399923, 528)


,1,g,r,i,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,airmass_u,airmass_i,class,diff_g_u,diff_r_g,diff_i_r,diff_z_i,diff_dered_u_z,diff_dered_g_dered_u,diff_dered_r_dered_g,diff_dered_i_dered_r,diff_dered_z_dered_i,diff_airmass_g_airmass_u,u_0,u_1,g_1,r_0,r_1,i_0,i_1,z_0,g^2,g r,g i,g redshift,g dered_u,g dered_g,g dered_r,g dered_i,g dered_z,g nObserve,g airmass_u,g airmass_i,g class,g diff_g_u,g diff_r_g,g diff_i_r,g diff_z_i,g diff_dered_u_z,...,diff_dered_z_dered_i r_0,diff_dered_z_dered_i r_1,diff_dered_z_dered_i i_0,diff_dered_z_dered_i i_1,diff_dered_z_dered_i z_0,diff_airmass_g_airmass_u^2,diff_airmass_g_airmass_u u_0,diff_airmass_g_airmass_u u_1,diff_airmass_g_airmass_u g_1,diff_airmass_g_airmass_u r_0,diff_airmass_g_airmass_u r_1,diff_airmass_g_airmass_u i_0,diff_airmass_g_airmass_u i_1,diff_airmass_g_airmass_u z_0,u_0^2,u_0 u_1,u_0 g_1,u_0 r_0,u_0 r_1,u_0 i_0,u_0 i_1,u_0 z_0,u_1^2,u_1 g_1,u_1 r_0,u_1 r_1,u_1 i_0,u_1 i_1,u_1 z_0,g_1^2,g_1 r_0,g_1 r_1,g_1 i_0,g_1 i_1,g_1 z_0,r_0^2,r_0 r_1,r_0 i_0,r_0 i_1,r_0 z_0,r_1^2,r_1 i_0,r_1 i_1,r_1 z_0,i_0^2,i_0 i_1,i_0 z_0,i_1^2,i_1 z_0,z_0^2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.1411,0.8707,0.2799,-0.1996,1.8259,1.2030,0.9113,0.3140,-0.0038,1.6086,0.1199,0.1201,0.0,-1.0412,-1.6279,-0.0460,0.3265,-2.0530,-1.2178,-1.7230,-0.0301,0.0339,-0.1271,-0.1876,-0.1880,-2.8983e-03,1.7535,1.1358,0.8638,0.2702,-0.0138,1.3020,0.9935,0.3194,-0.2277,2.0834,1.3727,1.0398,0.3583,-0.0044,1.8356,0.1369,0.1371,-2.1198,-1.1880,-1.8575,-0.0524,0.3725,-2.3426,...,0.0595,0.0385,0.0293,0.0092,-0.0005,0.0162,0.0238,0.0239,3.6836e-04,-0.2229,-0.1443,-0.1098,-0.0343,0.0017,0.0352,0.0353,5.4379e-04,-0.3290,-0.2131,-0.1621,-0.0507,0.0026,0.0353,5.4480e-04,-0.3296,-0.2135,-0.1624,-0.0508,0.0026,8.4002e-06,-0.0051,-3.2918e-03,-2.5037e-03,-7.8315e-04,3.9868e-05,3.0747,1.9915,1.5147,0.4738,-0.0241,1.2900,0.9811,0.3069,-0.0156,0.7462,0.2334,-0.0119,0.0730,-0.0037,0.0002
1,1.0,-2.6196,-2.9288,-3.0680,-0.1844,-2.4379,-2.5626,-2.8626,-3.0000,-0.2413,-0.8762,0.6658,0.6538,1.0,0.1440,0.4457,0.0115,-0.0679,0.5612,0.0932,0.4218,0.0091,0.8647,-0.2093,-0.2307,-0.2196,-2.7644e-03,-2.5193,-2.6757,-2.9882,-3.1356,-0.3598,6.8623,7.6724,8.0370,0.4831,6.3862,6.7129,7.4989,7.8588,0.6321,2.2953,-1.7441,-1.7128,-1.3923,-0.3772,-1.1676,-0.0301,0.1778,-1.4702,...,-2.1784,-2.3136,-2.5839,-2.7114,-0.3111,0.0438,0.0483,0.0460,5.7865e-04,0.5273,0.5601,0.6255,0.6564,0.0753,0.0532,0.0507,6.3771e-04,0.5812,0.6172,0.6893,0.7233,0.0830,0.0482,6.0702e-04,0.5532,0.5875,0.6562,0.6885,0.0790,7.6421e-06,0.0070,7.3968e-03,8.2607e-03,8.6683e-03,9.9467e-04,6.3467,6.7408,7.5281,7.8995,0.9065,7.1593,7.9955,8.3900,0.9627,8.9294,9.3700,1.0752,9.8322,1.1282,0.1295
2,1.0,-1.5627,-1.5040,-1.4525,-0.1977,-1.5801,-1.5497,-1.4816,-1.4278,-0.1053,-0.4287,-1.3182,-1.3312,0.0,0.6731,0.5198,0.0166,-0.1164,0.6173,0.7335,0.5280,0.0139,0.1901,-0.0676,-0.0679,-0.0656,-2.0309e-03,-1.5047,-1.4754,-1.4029,-1.3400,-0.1421,2.4420,2.3504,2.2699,0.3090,2.4692,2.4217,2.3153,2.2312,0.1645,0.6699,2.0599,2.0802,-0.8869,-1.0519,-0.8123,-0.0259,0.1819,-0.9646,...,-0.2860,-0.2805,-0.2667,-0.2547,-0.0270,0.0046,0.0046,0.0044,1.3732e-04,0.1017,0.0998,0.0949,0.0906,0.0096,0.0046,0.0045,1.3798e-04,0.1022,0.1002,0.0953,0.0910,0.0097,0.0043,1.3320e-04,0.0987,0.0968,0.0920,0.0879,0.0093,4.1247e-06,0.0031,2.9964e-03,2.8491e-03,2.7215e-03,2.8859e-04,2.2641,2.2200,2.1109,2.0163,0.2138,2.1768,2.0698,1.9771,0.2096,1.9680,1.8799,0.1993,1.7957,0.1904,0.0202
3,1.0,1.6517,1.6991,1.9694,-0.1993,2.9911,1.6467,1.6820,1.9443,0.1627,0.1351,0.2544,0.2566,0.0,-0.4472,0.1899,-0.0031,0.2846,-4.1016,-0.4874,0.2023,0.0004,-0.0246,0.1278,0.0425,0.0452,-1.0804e-03,2.9829,1.6385,1.6839,1.9532,0.2280,2.7280,2.8063,3.2528,-0.3291,4.9402,2.7198,2.7781,3.2114,0.2687,0.2232,0.4201,0.4239,-6.2154,-0.7386,0.3137,-0.0051,0.4701,-6.7745,...,-0.0733,-0.0403,-0.0414,-0.0480,-0.0056,0.0163,0.0054,0.0058,-1.3806e-04,0.3812,0.2094,0.2152,

In [ ]:
# train set
Ver1_X = dataset.loc[dataset[target_column] != -1 , :]
Ver1_X.drop(columns=target_column,inplace=True,axis=1)
Ver1_y = dataset.loc[dataset[target_column] != -1, target_column]
Ver1_y.astype(int)

# test set
Ver1_test = dataset.loc[dataset[target_column] == -1, :]
Ver1_test.drop(columns=target_column, inplace=True,axis=1)

print(Ver1_X.shape, Ver1_y.shape, Ver1_test.shape)

(319923, 527) (319923,) (80000, 527)


## Ver2 데이터 로드

In [ ]:
dataset = pd.read_csv(polynomial_feature_Ver2_file, index_col=0)
print(dataset.shape)
dataset.head()

(399923, 904)


,1,redshift,dered_g,dered_r,dered_i,airmass_g,ugriz_r,ugriz_i,airmass_diff,ugriz_z_ugriz_g_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_z_dered_g_diff,ugriz_i_ugriz_r_diff,ugriz_i_dered_g_diff,ugriz_r_ugriz_g_diff,ugriz_r_ugriz_u_diff,ugriz_r_dered_z_diff,ugriz_r_dered_r_diff,ugriz_r_dered_g_diff,ugriz_r_dered_u_diff,ugriz_g_ugriz_u_diff,ugriz_g_dered_g_diff,ugriz_g_dered_u_diff,ugriz_u_dered_r_diff,ugriz_u_dered_g_diff,dered_z_dered_i_diff,dered_z_airmass_g_diff,dered_r_dered_g_diff,dered_r_dered_u_diff,dered_g_dered_u_diff,dered_g_airmass_i_diff,dered_g_airmass_g_diff,dered_u_airmass_i_diff,airmass_z_airmass_g_diff,ugriz_dered_min_diff,dered_airmass_sum_diff,ugriz_icolor,ugriz_ucolor,ugriz_p1,dered_orthogonal,dered_parallel,redshift^2,redshift dered_g,redshift dered_r,redshift dered_i,redshift airmass_g,redshift ugriz_r,redshift ugriz_i,redshift airmass_diff,...,dered_g_airmass_g_diff ugriz_ucolor,dered_g_airmass_g_diff ugriz_p1,dered_g_airmass_g_diff dered_orthogonal,dered_g_airmass_g_diff dered_parallel,dered_u_airmass_i_diff^2,dered_u_airmass_i_diff airmass_z_airmass_g_diff,dered_u_airmass_i_diff ugriz_dered_min_diff,dered_u_airmass_i_diff dered_airmass_sum_diff,dered_u_airmass_i_diff ugriz_icolor,dered_u_airmass_i_diff ugriz_ucolor,dered_u_airmass_i_diff ugriz_p1,dered_u_airmass_i_diff dered_orthogonal,dered_u_airmass_i_diff dered_parallel,airmass_z_airmass_g_diff^2,airmass_z_airmass_g_diff ugriz_dered_min_diff,airmass_z_airmass_g_diff dered_airmass_sum_diff,airmass_z_airmass_g_diff ugriz_icolor,airmass_z_airmass_g_diff ugriz_ucolor,airmass_z_airmass_g_diff ugriz_p1,airmass_z_airmass_g_diff dered_orthogonal,airmass_z_airmass_g_diff dered_parallel,ugriz_dered_min_diff^2,ugriz_dered_min_diff dered_airmass_sum_diff,ugriz_dered_min_diff ugriz_icolor,ugriz_dered_min_diff ugriz_ucolor,ugriz_dered_min_diff ugriz_p1,ugriz_dered_min_diff dered_orthogonal,ugriz_dered_min_diff dered_parallel,dered_airmass_sum_diff^2,dered_airmass_sum_diff ugriz_icolor,dered_airmass_sum_diff ugriz_ucolor,dered_airmass_sum_diff ugriz_p1,dered_airmass_sum_diff dered_orthogonal,dered_airmass_sum_diff dered_parallel,ugriz_icolor^2,ugriz_icolor ugriz_ucolor,ugriz_icolor ugriz_p1,ugriz_icolor dered_orthogonal,ugriz_icolor dered_parallel,ugriz_ucolor^2,ugriz_ucolor ugriz_p1,ugriz_ucolor dered_orthogonal,ugriz_ucolor dered_parallel,ugriz_p1^2,ugriz_p1 dered_orthogonal,ugriz_p1 dered_parallel,dered_orthogonal^2,dered_orthogonal dered_parallel,dered_parallel^2,class
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,-0.1996,1.2030,0.9113,0.3140,0.1194,0.8707,0.2799,-0.8583,-0.1845,-0.3189,-2.8983e-03,-0.1929,-1.6279,-1.7477,-1.0412,-1.9084,0.0842,-0.1925,-1.1033,-1.9781,-1.8577,-0.1876,-1.7700,1.7208,1.4778,-0.0301,-0.0047,-1.2178,-2.0519,-2.0530,1.1988,1.1987,1.8252,-0.0333,-0.0043,0.4090,0.5061,-0.1287,1.9592,1.5056,1.8356,0.0398,-0.2401,-0.1819,-0.0627,-0.0238,-0.1737,-0.0559,0.1713,...,-0.1543,2.3484,1.8047,2.2003,3.3312,-0.0608,-0.0078,0.7465,0.9237,-0.2349,3.5758,2.7479,3.3503,0.0011,1.4154e-04,-0.0136,-0.0168,0.0043,-0.0652,-0.0501,-0.0611,1.8075e-05,-0.0017,-0.0022,0.0005,-0.0083,-6.4009e-03,-7.8041e-03,0.1673,0.2070,-0.0526,0.8013,0.6158,0.7508,0.2561,-0.0651,0.9916,0.7620,0.9290,0.0166,-0.2522,-0.1938,-0.2363,3.8384,2.9496,3.5963,2.2667,2.7636,3.3694,0.0
1,1.0,-0.1844,-2.5626,-2.8626,-3.0000,0.6881,-2.9288,-3.0680,0.4031,0.0417,0.0802,-2.7644e-03,0.0313,0.4457,0.2356,0.1440,0.4628,-0.0283,-0.2318,-0.0429,0.3392,0.5315,-0.2307,0.3268,-0.4957,-0.5716,0.0091,-0.2464,0.0932,0.4374,0.5612,-2.6172,-2.6206,-2.4863,-0.8601,-0.0040,-1.2258,-0.0250,0.2993,-0.5141,-0.4236,-0.3585,0.0340,0.4726,0.5279,0.5533,-0.1269,0.5402,0.5658,-0.0744,...,-0.7844,1.3473,1.1102,0.9394,6.1815,2.1386,0.0100,3.0476,0.0622,-0.7442,1.2782,1.0533,0.8912,0.7399,3.4617e-03,1.0544,0.0215,-0.2575,0.4422,0.3644,0.3083,1.6197e-05,0.0049,0.0001,-0.0012,0.0021,1.7050e-03,1.4426e-03,1.5025,0.0307,-0.3669,0.6302,0.5193,0.4394

In [ ]:
# train set
Ver2_X = dataset.loc[dataset[target_column] != -1 , :]
Ver2_X.drop(columns=target_column,inplace=True,axis=1)
Ver2_y = dataset.loc[dataset[target_column] != -1, target_column]
Ver2_y.astype(int)

# test set
Ver2_test = dataset.loc[dataset[target_column] == -1, :]
Ver2_test.drop(columns=target_column, inplace=True,axis=1)

print(Ver2_X.shape, Ver2_y.shape, Ver2_test.shape)

(319923, 903) (319923,) (80000, 903)


## Ver3 데이터 로드

In [ ]:
dataset = pd.read_csv(polynomial_feature_Ver3_file, index_col=0)
print(dataset.shape)
dataset.head()

In [ ]:
# train set
Ver3_X = dataset.loc[dataset[target_column] != -1 , :]
Ver3_X.drop(columns=target_column,inplace=True,axis=1)
Ver3_y = dataset.loc[dataset[target_column] != -1, target_column]
Ver3_y.astype(int)

# test set
Ver3_test = dataset.loc[dataset[target_column] == -1, :]
Ver3_test.drop(columns=target_column, inplace=True,axis=1)

print(Ver3_X.shape, Ver3_y.shape, Ver3_test.shape)

# 모델 학습 

## Light lgbm 

In [ ]:
# light gbm 
lgbm_params = {
    'num_threads': -1, # aliases: n_jobs
    'num_iterations': 100, # aliases: n_estimators
    'metric': 'multi_logloss',
    'learning_rate': 0.3, # aliases: eta
    'boosting': 'gbdt', # aliases: boosting_type
    'objective': 'multiclass', # aliases: softmax
    'num_class': 3,
    'random_state': SEED,
    'device_type': 'gpu', # aliases: device
    'gpu_use_dp': 'true',
    'verbosity': 0, # aliases: verbose
}

In [ ]:
mlogloss = []

lgbm_oof_pred_ver1 = np.zeros((Ver1_X.shape[0],num_class))
lgbm_test_pred_ver1 = np.zeros((Ver1_test.shape[0],num_class))
lgbm_oof_pred_ver2 = np.zeros((Ver2_X.shape[0],num_class))
lgbm_test_pred_ver2 = np.zeros((Ver2_test.shape[0],num_class))
lgbm_oof_pred_ver3 = np.zeros((Ver3_X.shape[0],num_class))
lgbm_test_pred_ver3 = np.zeros((Ver3_test.shape[0],num_class))

y = Ver1_y


for X, test in [(Ver1_X,Ver1_test), (Ver2_X,Ver2_test), (Ver3_X, Ver3_test)]:
    kFold = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)
    for fold, (trn_idx, val_idx) in enumerate(kFold.split(X,y)):
        X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]

        dtrain = lgbm.Dataset(X_train, label=y_train)
        dval = lgbm.Dataset(X_val, label=y_val)
        
        lgbm_clf = lgbm.train(params=lgbm_params, train_set=dtrain, num_boost_round=5000,valid_sets=[dtrain,dval], early_stopping_rounds=50, verbose_eval=5000)
        mlogloss.append(lgbm_clf.best_score['valid_1']['multi_logloss'])
        
        # Predict 
        if X.shape[1]==527:
            lgbm_test_pred_ver1 += lgbm_clf.predict(test) / (n_splits*4)
            lgbm_oof_pred_ver1[val_idx] += lgbm_clf.predict(X_val) / 4
        elif X.shape[1]==903:
            lgbm_test_pred_ver2 += lgbm_clf.predict(test) / (n_splits*4)
            lgbm_oof_pred_ver2[val_idx] += lgbm_clf.predict(X_val) / 4
        else:
            lgbm_test_pred_ver3 += lgbm_clf.predict(test) / (n_splits*4)
            lgbm_oof_pred_ver3[val_idx] += lgbm_clf.predict(X_val) / 4
        
    print("*"*100)
    print("Traning has finished")
print('lgbm ver1 logloss= ',log_loss(y,lgbm_oof_pred_ver1))
print('lgbm ver2 logloss= ',log_loss(y,lgbm_oof_pred_ver2))
print('lgbm ver3 logloss= ',log_loss(y,lgbm_oof_pred_ver3))

[13:21:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mlogloss:0.77303	val-mlogloss:0.77398
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.


# 제출 파일 및 기타 파일 생성

In [ ]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)

# Ver1 
sub[target_column] = np.argmax(lgbm_test_pred_ver1, axis=1)
sub.to_csv(lgbm_submission_ver1_file)

# Ver2
sub[target_column] = np.argmax(lgbm_test_pred_ver2, axis=1)
sub.to_csv(lgbm_submission_ver2_file)

# Ver3 
sub[target_column] = np.argmax(lgbm_test_pred_ver3, axis=1)
sub.to_csv(lgbm_submission_ver3_file)

In [ ]:
# lgbm_oof_pred 파일 생성

# Ver1
np.savetxt(lgbm_oof_pred_ver1_file, lgbm_oof_pred_ver1,fmt='%.18f',delimiter=',')

# Ver2
np.savetxt(lgbm_oof_pred_ver2_file, lgbm_oof_pred_ver2,fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(lgbm_oof_pred_ver3_file, lgbm_oof_pred_ver3,fmt='%.18f', delimiter=',')

In [ ]:
# lgbm_test_pred 파일 생성

# Ver1
np.savetxt(lgbm_test_pred_ver1_file, lgbm_test_pred_ver1,fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(lgbm_test_pred_ver2_file, lgbm_test_pred_ver2,fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(lgbm_test_pred_ver3_file, lgbm_test_pred_ver3,fmt='%.18f', delimiter=',')